In [1]:
from configparser import ConfigParser, ExtendedInterpolation
from collections import defaultdict
from itertools import combinations, product
import warnings, pandas as pd, nltk, string
warnings.filterwarnings('ignore')
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim, string
from gensim import corpora, models, similarities
from gensim.corpora import Dictionary
from gensim.matutils import kullback_leibler, hellinger
from gensim.models import ldamodel
from gensim.similarities import MatrixSimilarity, SparseMatrixSimilarity, Similarity
#from gensim.summarization.bm25 import get_bm25_weights
from gensim.utils import simple_preprocess
from IPython.core.display import display, HTML
from IPython.display import Image
from IPython.lib.display import YouTubeVideo
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from spacy.matcher import Matcher
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import NearestNeighbors
import spacy, string


In [10]:
df = pd.read_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/data_train.csv', encoding='utf-8')

In [11]:
data = df.iloc[:, 0].to_frame()
data = data.head(1)
data.head()

,source
0,Minghella war der Sohn italienisch-schottische...


In [12]:
stopword_list = nltk.corpus.stopwords.words('german')
def sentence_count(row):
    tokens = sent_tokenize(row, language='german')
    return tokens
data['source'] = data['source'].apply(sentence_count)

In [13]:
data.head()
data = data.explode('source').reset_index(drop=True)

In [15]:
def remove_stopwords(text):
    #text2 = text.translate(str.maketrans('', '', string.punctuation))
    filtered_words = [word for word in nltk.word_tokenize(text) if word not in stopword_list]
    filtered_text = ' '.join(filtered_words)
    string = ' '.join(filtered_text)
    return filtered_text

data["source"] = data["source"].apply(remove_stopwords)

In [16]:
data['source'][0]

'Minghella Sohn italienisch-schottischer Eltern , Isle of Wight Fabrik für Eiscreme betrieben .'

In [18]:
vect = CountVectorizer(max_features=10)
transform_vect = vect.fit_transform(data['source'])
index = ['doc_{}'.format(i) for i in range(23)]

dist_df = pd.DataFrame(
      transform_vect.todense()
    , columns=vect.get_feature_names()
    , index=index
)
dist_df

,ck,der,film,fu,minghella,oscar,regie,ta,tig,wurde
doc_0,0,0,0,1,1,0,0,0,0,0
doc_1,0,0,0,0,0,0,0,1,1,0
doc_2,0,0,0,0,0,0,0,0,0,0
doc_3,0,0,0,0,0,0,0,1,1,0
doc_4,0,0,1,1,0,0,0,0,0,1
doc_5,0,0,0,1,0,0,0,0,0,0
doc_6,0,2,0,3,0,1,0,0,0,1
doc_7,0,0,0,0,0,0,0,0,0,0
doc_8,0,1,0,0,1,1,1,0,0,0
doc_9,1,0,0,0,1,0,0,0,0,2


In [22]:
jaccard = pairwise_distances(dist_df.values, metric='jaccard')
print(jaccard)

[[0.         1.         1.         1.         0.75       0.5
  0.8        1.         0.8        0.75       1.         0.66666667
  0.5        0.5        0.8        0.5        0.5        0.33333333
  0.8        0.66666667 0.66666667 1.         0.5       ]
 [1.         0.         1.         0.         1.         1.
  1.         1.         1.         1.         1.         1.
  1.         1.         0.5        1.         1.         1.
  1.         1.         1.         1.         1.        ]
 [1.         1.         0.         1.         1.         1.
  1.         0.         1.         1.         0.         1.
  1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.        ]
 [1.         0.         1.         0.         1.         1.
  1.         1.         1.         1.         1.         1.
  1.         1.         0.5        1.         1.         1.
  1.         1.         1.         1.         1.        ]
 [0.75       1.         1.     

In [ ]:
def kl_divergence(p, q):
    return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))

In [ ]:
def lexScore(sim_matrix, dump, nodes, threshold):
    lst=[]
    rand = d/nodes
    for i in sim_matrix:
        for j in sim_matrix:
            if sim_matrix > trehsold:
                list[i]= rand+(1-d)*(sum(sim_matrix[i][j]/sum(g[j])))